In [0]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
train_df

In [3]:
print(train_df.shape)
print(test_df.shape)

(891, 12)
(418, 11)
